## StardewValley Wiki Scrapper



### Setup

In [20]:
%load_ext autoreload
%autoreload 2

import stars
from bs4 import BeautifulSoup
import requests as re
import pandas as pd
import regex

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Infox to DF

In [2]:
soup = BeautifulSoup(re.get('https://stardewvalleywiki.com/Fish_Taco').text)
table = soup.find('table', {'id':"infoboxtable"})
table_lines = table.find_all('tr')

In [24]:
data = {'lines': {}}

detail_flag = True
image_flag = True

for line in table_lines:
    
    # Finding the category the line is in
    if line.find('td', {"style":"text-align:center; padding-left: 0; padding-right: 0;"}) != None:
        cat_holder = line.find('td', {"style":"text-align:center; padding-left: 0; padding-right: 0;"}).text.replace('\n', '')

    # Finding the name of the item that the page is about
    elif line.find('td', {'id':"infoboxheader"}) != None:
        data['name'] = line.find('td', {'id':"infoboxheader"}).text.replace('\n', '')
    
    # Finding the image of the item that the page is about
    elif (line.find('img') != None) and (image_flag):
        data['image'] = 'https://stardewvalleywiki.com/' + line.find('img')['src']
        image_flag = False # This is to make sure that the image is only found once

    # Finding the description of the item that the page is about
    elif (line.find('td', {'id':"infoboxdetail"}) != None) and detail_flag:
        data['description'] = line.find('td', {'id':"infoboxdetail"}).text.replace('\n', '')
        detail_flag = False # This is to make sure that the description is only found once

    # Finding the contents of the infobox lines  
    elif line.find('td', {'id': 'infoboxsection'}) != None:
        section = line.find('td', {'id': 'infoboxsection'}).text.replace('\n', '')
        detail = stars.items.extract_dynamic(data, line)
        data['lines'][section] = detail
        

In [25]:
data

{'lines': {'Source': 'Cooking',
  'Buff(s)': {'Fishing': '+2'},
  'Buff Duration': ' 7m',
  'Energy / Health': {'Energy': '165', 'Health': '74'},
  'Sell Price': '500g',
  'Qi Seasoning': {'Energy': '297', 'Health': '133', 'Price': '750g'},
  'Recipe Source(s)': {'Source': 'Linus', 'Vehicle': 'Mail', 'When': '7+'},
  'Ingredients': {'Tuna': '1',
   'Tortilla': '1',
   'Red Cabbage': '1',
   'Mayonnaise': '1'}},
 'name': 'Fish Taco',
 'image': 'https://stardewvalleywiki.com//mediawiki/images/d/d5/Fish_Taco.png',
 'description': 'It smells delicious.'}

In [ ]:
data['lines']['Buff(s)']

' Fishing(+2)'

In [ ]:
regex.search(r'(.*)\(([+-][0-9])\)', data['lines']['Buff(s)']).group(2)

'+2'

In [26]:
pd.DataFrame(data)

,lines,name,image,description
Buff Duration,7m,Fish Taco,https://stardewvalleywiki.com//mediawiki/image...,It smells delicious.
Buff(s),{'Fishing': '+2'},Fish Taco,https://stardewvalleywiki.com//mediawiki/image...,It smells delicious.
Energy / Health,"{'Energy': '165', 'Health': '74'}",Fish Taco,https://stardewvalleywiki.com//mediawiki/image...,It smells delicious.
Ingredients,"{'Tuna': '1', 'Tortilla': '1', 'Red Cabbage': ...",Fish Taco,https://stardewvalleywiki.com//mediawiki/image...,It smells delicious.
Qi Seasoning,"{'Energy': '297', 'Health': '133', 'Price': '7...",Fish Taco,https://stardewvalleywiki.com//mediawiki/image...,It smells delicious.
Recipe Source(s),"{'Source': 'Linus', 'Vehicle': 'Mail', 'When':...",Fish Taco,https://stardewvalleywiki.com//mediawiki/image...,It smells delicious.
Sell Price,500g,Fish Taco,https://stardewvalleywiki.com//mediawiki/image...,It smells delicious.
Source,Cooking,Fish Taco,https://stardewvalleywiki.com//mediawiki/image...,It smells delicious.


In [2]:
!docker run \
    --name age  \
    -p 3000:3000 \
    -p 5455:5432 \
    -e POSTGRES_USER=postgresUser \
    -e POSTGRES_PASSWORD=postgresPW \
    -e POSTGRES_DB=postgresDB \
    -d \
    apache/age

docker: Error response from daemon: Conflict. The container name "/age" is already in use by container "9676ab66658eba74003b5fe38c7b698524e8da861a1c2d4151186bb5f28beee9". You have to remove (or rename) that container to be able to reuse that name.
See 'docker run --help'.
